In [9]:
import onnx
import torch.onnx
import numpy as np
import segmentation_models_pytorch as smp
test_model=torch.load('./timm-mobilenetv3_small_100_best_model1.pt', map_location='cpu')
# test_model.eval()
ENCODER = 'timm-mobilenetv3_small_100'
ENCODER_WEIGHTS = 'imagenet'
CLASSES = ['bacterial']
ACTIVATION = 'sigmoid'  # could be None for logits or 'softmax2d' for multiclass segmentation
DEVICE = 'cuda'


model = smp.Unet(
    encoder_name=ENCODER,
    encoder_weights=ENCODER_WEIGHTS,
    classes=1,
    activation=ACTIVATION,
)
model.load_state_dict(torch.load('./timm-mobilenetv3_small_100_best_model1.pt', map_location='cpu')).eval()

ModuleNotFoundError: No module named 'torch.fx'

In [10]:
onnx_model=onnx.load('./test_model.onnx')
onnx.checker.check_model(onnx_model)

In [11]:
x = torch.randn(1, 3, 512, 512, requires_grad=True)

In [12]:
import onnxruntime

ort_session = onnxruntime.InferenceSession("./test_model.onnx")

def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

# ONNX 런타임에서 계산된 결과값
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(x)}
ort_outs = ort_session.run(None, ort_inputs)

# ONNX 런타임과 PyTorch에서 연산된 결과값 비교
np.testing.assert_allclose(to_numpy(torch_out), ort_outs[0], rtol=1e-03, atol=1e-05)

print("Exported model has been tested with ONNXRuntime, and the result looks good!")

NameError: name 'torch_out' is not defined